In [1]:
import pickle
import numpy as np
import torch
import paddle
from paddle_net import FGCNN
from reprod_log import ReprodLogger, ReprodDiffHelper

torch版脚本链接: https://colab.research.google.com/drive/1D5xCfWJt3qvYsHtR1ZFSxHAqzwPYdTPI?usp=sharing

In [2]:
# load data
pipline_data = pickle.load(open('data/pipline_data.pickle', 'rb'))
pipline_label = pickle.load(open('data/pipline_label.pickle', 'rb'))

In [2]:
# paddle
sparse_inputs_slots = 27 - 1
sparse_feature_size = [27, 92, 172, 157, 12, 7, 183, 19, 2, 142, 173, 170, 166, 14, 170, 168, 9, 127, 44, 4, 169, 6, 10, 125, 20, 90]
sparse_feature_name = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']
sparse_feature_dim = 4

# sparse_inputs_slots = 14
# sparse_feature_size = [25, 95, 3, 48, 98, 97, 66, 10, 10, 3, 6, 3, 4, 3]
# sparse_feature_name = ['userid', 'adgroup_id', 'pid', 'cate_id', 'campaign_id', 'customer', 'brand', 'cms_segid', 'cms_group_id', 'final_gender_code', 'age_level', 'pvalue_level', 'shopping_level', 'occupation']

conv_kernel_width =  [7, 7, 7, 7]
conv_filters =  [14, 16, 18, 20]
new_maps =  [3, 3, 3, 3]
pooling_width =  [2, 2, 2, 2]
stride =  [1, 1]
dnn_hidden_units =  [128]
dnn_dropout =  0.0
model = FGCNN(sparse_inputs_slots, sparse_feature_size, sparse_feature_name, sparse_feature_dim, conv_kernel_width, 
                conv_filters, new_maps, pooling_width, stride, dnn_hidden_units, dnn_dropout)

[14, 16, 18, 20]


### step 1
deepctr的dnn层默认不使用bn, 论文里使用bn. 对齐时去掉了bn, 实际训练时使用bn

In [13]:
torch_key = \
    ['embedding_dict.C%d.weight' % (i+1) for i in range(26)] + \
    ['fg_embedding_dict.C%d.weight' % (i+1) for i in range(26)] + \
    ['fgcnn.conv_pooling.%d.0.weight' % i for i in range(4)] + \
    ['fgcnn.conv_pooling.%d.0.bias' % i for i in range(4)]

torch_linearkey = \
    ['fgcnn.recombination.%d.0.weight' % i for i in range(4)] + \
    ['fgcnn.recombination.%d.0.bias' % i for i in range(4)] + \
    ['dnn.linears.0.weight', 'dnn.linears.0.bias', 'dnn_linear.weight', 'out.bias']
    
paddle_key = \
    ['embedding.%d.embedding.weight' % i for i in range(26)] + \
    ['fg_embedding.%d.embedding.weight' % i for i in range(26)] + \
    ['fgcnn.conv_pooling.%d.0.weight' % i for i in range(4)] + \
    ['fgcnn.conv_pooling.%d.0.bias' % i for i in range(4)]
    
paddle_linearkey = \
    ['fgcnn.recombination.%d.0.weight' % i for i in range(4)]  + \
    ['fgcnn.recombination.%d.0.bias' % i for i in range(4)]  + \
    ['dnn.linears.0.0.weight', 'dnn.linears.0.0.bias', 'fc_linear.weight', 'fc_linear.bias']

key_map = {key_t:key_p for key_t, key_p in zip(torch_key, paddle_key)}
torch_para = torch.load('torch_para.pth')
paddle_para = {key_map[k]:paddle.to_tensor(v.numpy()) for k, v in torch_para.items() if k in torch_key}
for t, p in zip(torch_linearkey, paddle_linearkey):
    paddle_para[p] = paddle.to_tensor(torch_para[t].T.numpy())
model.set_state_dict(paddle_para)

In [5]:
model.eval()
forward = []
for data in pipline_data:
    out = model.forward(paddle.to_tensor(data[0, :26]))
    forward.append(out.numpy())
reprod_logger = ReprodLogger()
reprod_logger.add("logits", np.array(forward))
reprod_logger.save("step1/forward_paddle.npy")

### step 2

In [26]:
cost = []
for inputs, label in zip(pipline_data, pipline_label):
    out = model.forward(inputs[0, :26])
    cost.append(paddle.nn.functional.binary_cross_entropy(out, paddle.to_tensor(float(label)).reshape((1,1))).numpy()[0])
reprod_logger = ReprodLogger()
reprod_logger.add("binary_cross_entropy", np.array(cost))
reprod_logger.save("step2/metric_paddle.npy")

### step 3

In [52]:
test_pred = [0.9, 0.8, 0.3, 0.1, 0.4, 0.9, 0.66, 0.7]
test_label = [1,0,0,0,1,0,1,0]

In [53]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_label, test_pred)

0.5666666666666667

In [60]:
preds = []
for pred in test_pred:
    # pred = model.forward(inputs[0, :26])
    pred = paddle.to_tensor(pred)
    predict_2d = paddle.concat(x=[1 - pred, pred])
    preds.append(predict_2d)
auc = paddle.metric.Auc("ROC")
auc.update(preds=np.array(preds).reshape(-1, 2), labels=np.array(test_label).astype('float'))
auc.accumulate()

0.5666666666666667

### step 4

In [19]:
model.set_state_dict(paddle_para)
model.train()
optimizer = paddle.optimizer.Adam(
            parameters=model.parameters(),
            learning_rate=0.001)
loss_list = []
for inputs, label in zip(pipline_data, pipline_label):
    optimizer.clear_grad()
    pred = model.forward(inputs[0, :26])
    loss = paddle.nn.functional.binary_cross_entropy(pred, paddle.to_tensor(float(label)).reshape((1,1)))
    loss_list.append(loss.numpy())
    loss.backward()
    optimizer.step()
reprod_logger = ReprodLogger()
for i in range(len(loss_list)):
    reprod_logger.add("loss_%d" % i,loss_list[i])
reprod_logger.save("step4/bp_align_paddle.npy")

### step 5

In [30]:
model.set_state_dict(paddle_para)
model.train()
optimizer = paddle.optimizer.Adam(
            parameters=model.parameters(),
            learning_rate=0.001)
loss_list = []
for epoch in range(2):
    for inputs, label in zip(pipline_data, pipline_label):
        optimizer.clear_grad()
        pred = model.forward(inputs[0, :26])
        loss = paddle.nn.functional.binary_cross_entropy(pred, paddle.to_tensor(float(label)).reshape((1,1)))
        loss_list.append(loss.numpy()[0])
        loss.backward()
        optimizer.step()
reprod_logger = ReprodLogger()
reprod_logger.add("train", np.array(loss_list))
reprod_logger.save("step5/train_paddle.npy")

### check diff

In [7]:
# step1
diff_helper = ReprodDiffHelper()
torch_info = diff_helper.load_info("step1/forward_torch.npy")
paddle_info = diff_helper.load_info("step1/forward_paddle.npy")

diff_helper.compare_info(torch_info, paddle_info)

diff_helper.report(path="diff/forward_diff.log")

[2021/12/26 19:14:53] root INFO: logits: 
[2021/12/26 19:14:53] root INFO: 	mean diff: check passed: True, value: 0.0
[2021/12/26 19:14:53] root INFO: diff check passed


In [5]:
# step 2
diff_helper = ReprodDiffHelper()
torch_info = diff_helper.load_info("step2/metric_torch.npy")
paddle_info = diff_helper.load_info("step2/metric_paddle.npy")

diff_helper.compare_info(torch_info, paddle_info)

diff_helper.report(path="diff/metric_diff.log")

[2021/12/26 20:30:59] root INFO: binary_cross_entropy: 
[2021/12/26 20:30:59] root INFO: 	mean diff: check passed: True, value: 0.0
[2021/12/26 20:30:59] root INFO: diff check passed


In [56]:
# step 3

In [4]:
# step 4
diff_helper = ReprodDiffHelper()
torch_info = diff_helper.load_info("step4/bp_align_torch.npy")
paddle_info = diff_helper.load_info("step4/bp_align_paddle.npy")

diff_helper.compare_info(torch_info, paddle_info)

diff_helper.report(path="diff/bp_align_diff.log")
print('误差: %.6f' % (0.000389695 / 1.0492053 * 10) + '%')

[2021/12/27 19:39:09] root INFO: loss_0: 
[2021/12/27 19:39:09] root INFO: 	mean diff: check passed: True, value: 0.0
[2021/12/27 19:39:09] root INFO: loss_1: 
[2021/12/27 19:39:09] root INFO: 	mean diff: check passed: False, value: 1.7881393432617188e-06
[2021/12/27 19:39:09] root INFO: loss_2: 
[2021/12/27 19:39:09] root INFO: 	mean diff: check passed: False, value: 1.8864870071411133e-05
[2021/12/27 19:39:09] root INFO: loss_3: 
[2021/12/27 19:39:09] root INFO: 	mean diff: check passed: False, value: 3.838539123535156e-05
[2021/12/27 19:39:09] root INFO: loss_4: 
[2021/12/27 19:39:09] root INFO: 	mean diff: check passed: False, value: 2.6181340217590332e-05
[2021/12/27 19:39:09] root INFO: loss_5: 
[2021/12/27 19:39:09] root INFO: 	mean diff: check passed: False, value: 0.0007915496826171875
[2021/12/27 19:39:09] root INFO: loss_6: 
[2021/12/27 19:39:09] root INFO: 	mean diff: check passed: False, value: 2.024136483669281e-05
[2021/12/27 19:39:09] root INFO: loss_7: 
[2021/12/27 19:

In [3]:
# step 5
diff_helper = ReprodDiffHelper()
torch_info = diff_helper.load_info("step5/train_torch.npy")
paddle_info = diff_helper.load_info("step5/train_paddle.npy")

diff_helper.compare_info(torch_info, paddle_info)

diff_helper.report(path="diff/loss_diff.log")
print('误差: %.6f' % (0.0003 / 0.69180256 * 10) + '%')

[2021/12/27 19:39:26] root INFO: train: 
[2021/12/27 19:39:26] root INFO: 	mean diff: check passed: False, value: 0.00037922762567177415
[2021/12/27 19:39:26] root INFO: diff check failed
误差: 0.004336%


In [ ]:
import fuxictr # python 3.6.*

: 